In [1]:
import MVTecAD_loaddata as mv
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import csv
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras import optimizers
from efficientnet.tfkeras import EfficientNetB2
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input, BatchNormalization, LeakyReLU, Activation, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import load_img, img_to_array, array_to_img
# from keras.utils.vis_utils import plot_model
import random 

In [22]:
def OCSVM(X, x_test, y, y_test):
    tuned_parameters = [
        {'nu': [2**-3], 'kernel': ['rbf'], 'gamma': [10**-4, 10**-5, 10**-6]},
    ]
    score = 'roc_auc_ovr_weighted'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
    clf.fit(X, y)
    a = clf.decision_function(x_test)
    b = clf.predict(x_test)
    r1 = roc_auc_score(y_test,a)
    r2 = f1_score(y_test,b)
    r3 = accuracy_score(y_test,b)
    print('roc_auc : ',r1)
    print('f1_score : ', r2)
    print('accuracy_score : ', r3)
    print('best_params : ', clf.best_params_)
    result = [r1,r2,r3]
    return result

def model_predict(X_train, X_test):
    input_shape = (256, 256, 3)
    inputs = Input(input_shape)
    base_model = EfficientNetB2(include_top=False, weights='imagenet')(inputs)
    model = Model(inputs=inputs, outputs=base_model)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss="mean_squared_error")
    f_tr = model.predict(X_train, verbose=0)
    f_ts = model.predict(X_test, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], f_tr.shape[1] * f_tr.shape[2] * f_tr.shape[3]))
    x_ts = np.reshape(f_ts, (f_ts.shape[0], f_ts.shape[1] * f_ts.shape[2] * f_ts.shape[3]))
    return x_tr, x_ts

def OCSVM_SV(X, y, train_data):
    tuned_parameters = [
    {'nu': [2**-3, 2**-4, 2**-5, 2**-6], 'kernel': ['rbf'], 'gamma': [10**-3, 10**-4, 10**-5, 10**-6, 10**-7]},
    ]
    score = 'roc_auc_ovr_weighted'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
    clf.fit(X, y)
    ocsvm = OneClassSVM(nu=clf.best_params_['nu'], gamma=clf.best_params_['gamma'])
    ocsvm.fit(X,y)
    sv_num = ocsvm.support_
    sv = train_data[sv_num,:]
    print('best_params : ', clf.best_params_)
    return clf, ocsvm, sv

In [3]:
def train_gan_data(X_train):
    # ハイパーパラメータの設定
    # バッチサイズ
    batch_size = 16
    # エポック数
    epochs = 25
    # 学習率
    lr = 0.001

    # Generatorモデルの構築
    def build_generator():
        # Generatorは、潜在変数からRGB画像を生成するモデルです。
        # Conv2DTranspose層とBatchNormalization層とLeakyReLU層を用いて、アップサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Flatten(input_shape=(256, 256, 3)))
#         model.add(Dense(8*8*512, input_dim=(256, 256, 3)))
        model.add(Reshape((16, 16, 768)))
        # 中間層1
        model.add(Conv2DTranspose(256, kernel_size=5, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層4
        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
        model.summary()
        return model

    # Discriminatorモデルの構築
    def build_discriminator():
        # Discriminatorは、RGB画像が本物か偽物かを判定するモデルです。
        # Conv2D層とBatchNormalization層とLeakyReLU層を用いて、ダウンサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(256, 256 ,3)))
        model.add(LeakyReLU(alpha=0.2))
        # 中間層1
        model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        return model

    # Generatorモデルの作成
    generator = build_generator()
    # Discriminatorモデルの作成
    discriminator = build_discriminator()
    # Discriminatorモデルのコンパイル
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
    # Discriminatorモデルの学習を停止
    discriminator.trainable = False

    # GANモデルの構築
    # GANモデルは、GeneratorモデルとDiscriminatorモデルを結合したモデルです。
    # 潜在変数を入力として、Discriminatorモデルの出力を得るようにします。
    gan_input = Input(shape=(256, 256, 3))
    gan_output = discriminator(generator(gan_input))
    gan = Model(gan_input, gan_output)
    # GANモデルのコンパイル
    gan.compile(loss='binary_crossentropy', optimizer=Adam(lr))

    # 学習のループ
    for epoch in range(epochs):
        # バッチごとに学習
        for i in range(0, len(X_train), batch_size):
            # 本物の画像データをバッチサイズ分取得
            real_images = X_train[i:i+batch_size]
            # 偽物の画像データをバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), 256, 256, 3))
            fake_images = generator.predict(latent_vectors)
            # 本物の画像データと偽物の画像データを結合
            images = np.concatenate([real_images, fake_images])
            # 本物の画像データにはラベル1を、偽物の画像データにはラベル0を付ける
            labels = np.concatenate([np.ones((len(real_images), 1)), np.zeros((len(real_images), 1))])
            # ラベルにわずかなノイズを加える
#             labels += 0.05 * np.random.random(labels.shape)
            # Discriminatorモデルの学習を再開
            discriminator.trainable = True
            # Discriminatorモデルの学習と評価
            loss1, acc1 = discriminator.train_on_batch(images, labels)
            # Discriminatorモデルの学習を停止
            discriminator.trainable = False
            # 潜在変数をバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), 256, 256, 3))
            # 偽物の画像データにはラベル1を付ける（騙す）
            misleading_labels = np.ones((len(real_images), 1))
            # GANモデルの学習
            gan.train_on_batch(latent_vectors, misleading_labels)
#             gan.train_on_batch(images, labels)
        # エポックごとに結果を表示
        print(f'Epoch {epoch}, Loss: {loss1}, Accuracy: {acc1}')
#         print(f'Epoch {epoch}, Loss: {loss2}, Accuracy: {acc2}')
    imageslatent_vectors = np.random.normal(0, 1, size=(int(len(X_train)), 256, 256, 3))
    fake_images = generator.predict(imageslatent_vectors)
    return fake_images, generator
def sv_mask(X):
    
    X_list = []
    for i in range(len(X)):
        for j in range(2):
            rand_x = random.randint(32,224)
            rand_y = random.randint(32,224)
            x1, x2 = rand_x-20, rand_x+20
            y1, y2 = rand_y-20, rand_y+20
            X[i,x1:x2, y1:y2, :] = 1.0
            X_list.append(X[i])
    X_list = np.array(X_list)*255
    return X_list

def save_svmask(img, name):
    for i in range(len(img)):
        cv2.imwrite('sv_fakes_train/'+name+'/'+str(i)+'.png', img[i])

def make_dataset(fake_image, train_data, y_train):
    fake_image = fake_image[:int(len(train_data)*0.125)]/255
    x_train = np.concatenate([train_data, fake_image])
    y = []
    for i in range(len(fake_image)):
        y.append(-1)
    y = np.array(y)
    y = y.reshape([len(y), 1])
    labels = np.concatenate([y_train, y])
    return x_train, labels
res = []

In [4]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'bottle')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

best_params :  {'gamma': 0.001, 'kernel': 'rbf', 'nu': 0.125}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 196608)            0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 768)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 256)      4915456   
 nspose)                                                         
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 256)      1024      
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16, 16, 256)       0         
                                                            

1/1 [==============================] - 1s 658ms/step
Epoch 17, Loss: 4.446144494241189e-11, Accuracy: 1.0
1/1 [==============================] - 1s 651ms/step
Epoch 18, Loss: 4.503243958287051e-11, Accuracy: 1.0
1/1 [==============================] - 1s 595ms/step
Epoch 19, Loss: 4.5803583148540383e-11, Accuracy: 1.0
1/1 [==============================] - 1s 631ms/step
Epoch 20, Loss: 4.6418778543166894e-11, Accuracy: 1.0
1/1 [==============================] - 1s 609ms/step
Epoch 21, Loss: 4.678155085535707e-11, Accuracy: 1.0
1/1 [==============================] - 1s 596ms/step
Epoch 22, Loss: 4.726735669535742e-11, Accuracy: 1.0
1/1 [==============================] - 1s 596ms/step
Epoch 23, Loss: 4.7721680773715747e-11, Accuracy: 1.0
1/1 [==============================] - 1s 631ms/step
Epoch 24, Loss: 4.8257432772036424e-11, Accuracy: 1.0
7/7 [==============================] - 14s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9920634920634921
f1_score :  0.7647058823529412
accuracy_score :  0.9036144578313253
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [5]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'cable')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 1s 926ms/step
Epoch 8, Loss: 2.2470693043612637e-36, Accuracy: 1.0
1/1 [==============================] - 1s 923ms/step
Epoch 9, Loss: 2.2237081119274644e-36, Accuracy: 1.0
1/1 [==============================] - 1s 903ms/step
Epoch 10, Loss: 2.2417503789646445e-36, Accuracy: 1.0
1/1 [==============================] - 1s 908ms/step
Epoch 11, Loss: 2.244768035971211e-36, Accuracy: 1.0
1/1 [==============================] - 1s 898ms/step
Epoch 12, Loss: 2.251934433263196e-36, Accuracy: 1.0
1/1 [==============================] - 1s 899ms/step
Epoch 13, Loss: 2.2510415483025037e-36, Accuracy: 1.0
1/1 [==============================] - 1s 924ms/step
Epoch 14, Loss: 2.259590500290555e-36, Accuracy: 1.0
1/1 [==============================] - 1s 908ms/step
Epoch 15, Loss: 2.2536904283948108e-36, Accuracy: 1.0
1/1 [==============================] - 1s 911ms/step
Epoch 16, Loss: 2.2608376335030285e-36, Accuracy: 1.0
1/1 [==============================] - 1s 

1/1 [==============================] - 1s 924ms/step
Epoch 19, Loss: 2.2675733725405696e-36, Accuracy: 1.0
1/1 [==============================] - 1s 907ms/step
Epoch 20, Loss: 2.2762154155882028e-36, Accuracy: 1.0
1/1 [==============================] - 1s 928ms/step
Epoch 21, Loss: 2.2792235661859873e-36, Accuracy: 1.0
1/1 [==============================] - 1s 908ms/step
Epoch 22, Loss: 2.2919967716311026e-36, Accuracy: 1.0
1/1 [==============================] - 1s 910ms/step
Epoch 23, Loss: 2.3008526191932286e-36, Accuracy: 1.0
1/1 [==============================] - 1s 909ms/step
Epoch 24, Loss: 2.3036263381631254e-36, Accuracy: 1.0
14/14 [==============================] - 31s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.930613961312027
f1_score :  0.78
accuracy_score :  0.8428571428571429
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [6]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'capsule')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

best_params :  {'gamma': 0.001, 'kernel': 'rbf', 'nu': 0.125}
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 196608)            0         
                                                                 
 reshape_2 (Reshape)         (None, 16, 16, 768)       0         
                                                                 
 conv2d_transpose_10 (Conv2D  (None, 16, 16, 256)      4915456   
 Transpose)                                                      
                                                                 
 batch_normalization_14 (Bat  (None, 16, 16, 256)      1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                          

1/1 [==============================] - 0s 377ms/step
Epoch 11, Loss: 3.975713417476886e-24, Accuracy: 1.0
1/1 [==============================] - 0s 360ms/step
Epoch 12, Loss: 3.980331803646503e-24, Accuracy: 1.0
1/1 [==============================] - 0s 365ms/step
Epoch 13, Loss: 3.990863096731203e-24, Accuracy: 1.0
1/1 [==============================] - 0s 376ms/step
Epoch 14, Loss: 4.005517371337041e-24, Accuracy: 1.0
1/1 [==============================] - 0s 372ms/step
Epoch 15, Loss: 4.0157784795617036e-24, Accuracy: 1.0
1/1 [==============================] - 0s 369ms/step
Epoch 16, Loss: 4.0243123768343845e-24, Accuracy: 1.0
1/1 [==============================] - 0s 361ms/step
Epoch 17, Loss: 4.0279316706675385e-24, Accuracy: 1.0
1/1 [==============================] - 0s 388ms/step
Epoch 18, Loss: 4.030191362730544e-24, Accuracy: 1.0
1/1 [==============================] - 0s 364ms/step
Epoch 19, Loss: 4.031519015634031e-24, Accuracy: 1.0
1/1 [==============================] - 0s 3

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.7526924611088951
f1_score :  0.391304347826087
accuracy_score :  0.5757575757575758
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [7]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'carpet')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

best_params :  {'gamma': 0.001, 'kernel': 'rbf', 'nu': 0.125}
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 196608)            0         
                                                                 
 reshape_3 (Reshape)         (None, 16, 16, 768)       0         
                                                                 
 conv2d_transpose_15 (Conv2D  (None, 16, 16, 256)      4915456   
 Transpose)                                                      
                                                                 
 batch_normalization_21 (Bat  (None, 16, 16, 256)      1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                          

1/1 [==============================] - 0s 362ms/step
Epoch 12, Loss: 3.4655972210896613e-13, Accuracy: 1.0
1/1 [==============================] - 0s 374ms/step
Epoch 13, Loss: 3.460917804513214e-13, Accuracy: 1.0
1/1 [==============================] - 0s 364ms/step
Epoch 14, Loss: 3.459287977597425e-13, Accuracy: 1.0
1/1 [==============================] - 0s 376ms/step
Epoch 15, Loss: 3.4610929031640703e-13, Accuracy: 1.0
1/1 [==============================] - 0s 375ms/step
Epoch 16, Loss: 3.458345534858992e-13, Accuracy: 1.0
1/1 [==============================] - 0s 375ms/step
Epoch 17, Loss: 3.452211661068155e-13, Accuracy: 1.0
1/1 [==============================] - 0s 375ms/step
Epoch 18, Loss: 3.4395966977407433e-13, Accuracy: 1.0
1/1 [==============================] - 0s 390ms/step
Epoch 19, Loss: 3.444377216169775e-13, Accuracy: 1.0
1/1 [==============================] - 0s 376ms/step
Epoch 20, Loss: 3.455549648506695e-13, Accuracy: 1.0
1/1 [==============================] - 0s 3

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.904431216931217
f1_score :  0.5862068965517241
accuracy_score :  0.8235294117647058
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [8]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'grid')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 0s 391ms/step
Epoch 16, Loss: 2.726928782341505e-28, Accuracy: 1.0
1/1 [==============================] - 0s 371ms/step
Epoch 17, Loss: 2.7286471933343847e-28, Accuracy: 1.0
1/1 [==============================] - 0s 378ms/step
Epoch 18, Loss: 2.733314443813364e-28, Accuracy: 1.0
1/1 [==============================] - 0s 379ms/step
Epoch 19, Loss: 2.748885587413735e-28, Accuracy: 1.0
1/1 [==============================] - 0s 359ms/step
Epoch 20, Loss: 2.7525826506832292e-28, Accuracy: 1.0
1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 0s 359ms/step
Epoch 21, Loss: 2.760012888408675e-28, Accuracy: 1.0
1/1 [==============================] - 0s 367ms/step
Epoch 22, Loss: 2.789117541728255e-28, Accuracy: 1.0
1/1 [==============================] - 0s 362ms/step
Epoch 23, Loss: 2.795850111331347e-28, Accuracy: 1.0
1/1 [==============================] - 0s 374ms/step
Epoch 24, Loss: 2.803937091167829e-28, Accuracy: 1.0
16/16 [==============================] - 29s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.5371762740183793
f1_score :  0.39285714285714285
accuracy_score :  0.5641025641025641
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [9]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'hazelnut')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 1s 803ms/step
Epoch 17, Loss: 1.9662226014016682e-10, Accuracy: 1.0
1/1 [==============================] - 1s 802ms/step
Epoch 18, Loss: 1.7969892240898844e-10, Accuracy: 1.0
1/1 [==============================] - 1s 803ms/step
Epoch 19, Loss: 1.64384797440853e-10, Accuracy: 1.0
1/1 [==============================] - 1s 810ms/step
Epoch 20, Loss: 1.56082410884828e-10, Accuracy: 1.0
1/1 [==============================] - 1s 795ms/step
Epoch 21, Loss: 1.4760984101691577e-10, Accuracy: 1.0
1/1 [==============================] - 1s 797ms/step
Epoch 22, Loss: 1.3717159352832908e-10, Accuracy: 1.0
1/1 [==============================] - 1s 940ms/step


1/1 [==============================] - 1s 862ms/step
Epoch 23, Loss: 1.3558208722397325e-10, Accuracy: 1.0
1/1 [==============================] - 1s 791ms/step
Epoch 24, Loss: 1.274055166922139e-10, Accuracy: 1.0
25/25 [==============================] - 48s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9128571428571429
f1_score :  0.392156862745098
accuracy_score :  0.7181818181818181
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [10]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'leather')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

best_params :  {'gamma': 0.001, 'kernel': 'rbf', 'nu': 0.125}
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_12 (Flatten)        (None, 196608)            0         
                                                                 
 reshape_6 (Reshape)         (None, 16, 16, 768)       0         
                                                                 
 conv2d_transpose_30 (Conv2D  (None, 16, 16, 256)      4915456   
 Transpose)                                                      
                                                                 
 batch_normalization_42 (Bat  (None, 16, 16, 256)      1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_48 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                         

Epoch 13, Loss: 2.7661406151735513e-11, Accuracy: 1.0
1/1 [==============================] - 1s 813ms/step
Epoch 14, Loss: 2.815998649541296e-11, Accuracy: 1.0
1/1 [==============================] - 1s 793ms/step
Epoch 15, Loss: 2.851298884498643e-11, Accuracy: 1.0
1/1 [==============================] - 1s 818ms/step
Epoch 16, Loss: 2.900803208749636e-11, Accuracy: 1.0
1/1 [==============================] - 1s 816ms/step
Epoch 17, Loss: 2.9461946771114356e-11, Accuracy: 1.0
1/1 [==============================] - 1s 803ms/step
Epoch 18, Loss: 2.99999539099538e-11, Accuracy: 1.0
1/1 [==============================] - 1s 812ms/step
Epoch 19, Loss: 3.0539019230113595e-11, Accuracy: 1.0
1/1 [==============================] - 1s 816ms/step
Epoch 20, Loss: 3.113613880612043e-11, Accuracy: 1.0
1/1 [==============================] - 1s 800ms/step
Epoch 21, Loss: 3.1717711790335557e-11, Accuracy: 1.0
1/1 [==============================] - 1s 818ms/step
Epoch 22, Loss: 3.249272725880381e-11, Accu

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9877717391304348
f1_score :  0.7307692307692308
accuracy_score :  0.8870967741935484
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [11]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'metal_nut')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 0s 486ms/step
Epoch 8, Loss: 6.623295245589134e-10, Accuracy: 1.0
1/1 [==============================] - 0s 471ms/step
Epoch 9, Loss: 5.767956667845908e-10, Accuracy: 1.0
1/1 [==============================] - 0s 470ms/step
Epoch 10, Loss: 5.084853649250931e-10, Accuracy: 1.0
1/1 [==============================] - 0s 466ms/step
Epoch 11, Loss: 4.541417242265311e-10, Accuracy: 1.0
1/1 [==============================] - 1s 508ms/step
Epoch 12, Loss: 4.100050299271629e-10, Accuracy: 1.0
1/1 [==============================] - 0s 485ms/step
Epoch 13, Loss: 3.742152143715316e-10, Accuracy: 1.0
1/1 [==============================] - 0s 485ms/step
Epoch 14, Loss: 3.462294895228979e-10, Accuracy: 1.0
1/1 [==============================] - 0s 487ms/step
Epoch 15, Loss: 3.216535093830686e-10, Accuracy: 1.0
1/1 [==============================] - 0s 472ms/step
Epoch 16, Loss: 2.985560410007082e-10, Accuracy: 1.0
1/1 [==============================] - 0s 486ms/

1/1 [==============================] - 0s 466ms/step
Epoch 19, Loss: 2.430973200517883e-10, Accuracy: 1.0
1/1 [==============================] - 1s 501ms/step
Epoch 20, Loss: 2.2844512392872218e-10, Accuracy: 1.0
1/1 [==============================] - 0s 480ms/step
Epoch 21, Loss: 2.1482277068329836e-10, Accuracy: 1.0
1/1 [==============================] - 0s 471ms/step
Epoch 22, Loss: 2.0236028130948824e-10, Accuracy: 1.0
1/1 [==============================] - 0s 471ms/step
Epoch 23, Loss: 1.9108745141771522e-10, Accuracy: 1.0
1/1 [==============================] - 0s 494ms/step
Epoch 24, Loss: 1.807793081898268e-10, Accuracy: 1.0
14/14 [==============================] - 25s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9354838709677419
f1_score :  0.6511627906976744
accuracy_score :  0.8695652173913043
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [12]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'pill')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 0s 365ms/step
Epoch 16, Loss: 1.005472038961841e-09, Accuracy: 1.0
1/1 [==============================] - 0s 376ms/step
Epoch 17, Loss: 9.288116942229863e-10, Accuracy: 1.0
1/1 [==============================] - 0s 372ms/step
Epoch 18, Loss: 8.613831314896458e-10, Accuracy: 1.0
1/1 [==============================] - 0s 360ms/step
Epoch 19, Loss: 8.005139329192446e-10, Accuracy: 1.0
1/1 [==============================] - 0s 391ms/step
Epoch 20, Loss: 7.463027418275203e-10, Accuracy: 1.0
1/1 [==============================] - 0s 364ms/step
Epoch 21, Loss: 6.976474398179278e-10, Accuracy: 1.0
1/1 [==============================] - 0s 364ms/step
Epoch 22, Loss: 6.541405195292782e-10, Accuracy: 1.0
1/1 [==============================] - 0s 367ms/step
Epoch 23, Loss: 6.148578868270249e-10, Accuracy: 1.0
1/1 [==============================] - 1s 924ms/step


1/1 [==============================] - 0s 364ms/step
Epoch 24, Loss: 5.791521151543577e-10, Accuracy: 1.0
17/17 [==============================] - 31s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.7119476268412439
f1_score :  0.3669724770642202
accuracy_score :  0.5868263473053892
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [13]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'screw')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 1s 908ms/step
Epoch 13, Loss: 2.659128472970451e-10, Accuracy: 1.0
1/1 [==============================] - 1s 906ms/step
Epoch 14, Loss: 2.3841503771215855e-10, Accuracy: 1.0
1/1 [==============================] - 1s 903ms/step
Epoch 15, Loss: 2.1860893650860191e-10, Accuracy: 1.0
1/1 [==============================] - 1s 903ms/step
Epoch 16, Loss: 2.043570312970644e-10, Accuracy: 1.0
1/1 [==============================] - 1s 927ms/step
Epoch 17, Loss: 1.894496781673638e-10, Accuracy: 1.0
1/1 [==============================] - 1s 912ms/step
Epoch 18, Loss: 1.7551868292109418e-10, Accuracy: 1.0
1/1 [==============================] - 1s 899ms/step
Epoch 19, Loss: 1.6125198398775353e-10, Accuracy: 1.0
1/1 [==============================] - 1s 924ms/step
Epoch 20, Loss: 1.5216969351250498e-10, Accuracy: 1.0
1/1 [==============================] - 1s 900ms/step


1/1 [==============================] - 1s 986ms/step
Epoch 21, Loss: 1.4248577318021205e-10, Accuracy: 1.0
1/1 [==============================] - 1s 913ms/step
Epoch 22, Loss: 1.3192566483688495e-10, Accuracy: 1.0
1/1 [==============================] - 1s 916ms/step
Epoch 23, Loss: 1.2194030507561848e-10, Accuracy: 1.0
1/1 [==============================] - 1s 907ms/step
Epoch 24, Loss: 1.1351429518580147e-10, Accuracy: 1.0
20/20 [==============================] - 37s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.8534535765525721
f1_score :  0.4999999999999999
accuracy_score :  0.8125
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [14]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'tile')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

best_params :  {'gamma': 0.001, 'kernel': 'rbf', 'nu': 0.125}
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_20 (Flatten)        (None, 196608)            0         
                                                                 
 reshape_10 (Reshape)        (None, 16, 16, 768)       0         
                                                                 
 conv2d_transpose_50 (Conv2D  (None, 16, 16, 256)      4915456   
 Transpose)                                                      
                                                                 
 batch_normalization_70 (Bat  (None, 16, 16, 256)      1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_80 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                         

1/1 [==============================] - 1s 691ms/step
Epoch 13, Loss: 5.868891744115201e-12, Accuracy: 1.0
1/1 [==============================] - 1s 692ms/step
Epoch 14, Loss: 5.8648901707369916e-12, Accuracy: 1.0
1/1 [==============================] - 1s 705ms/step
Epoch 15, Loss: 5.864948717654306e-12, Accuracy: 1.0
1/1 [==============================] - 1s 702ms/step
Epoch 16, Loss: 5.869270347513833e-12, Accuracy: 1.0
1/1 [==============================] - 1s 693ms/step
Epoch 17, Loss: 5.869931710839049e-12, Accuracy: 1.0
1/1 [==============================] - 1s 687ms/step
Epoch 18, Loss: 5.866428870460183e-12, Accuracy: 1.0
1/1 [==============================] - 1s 692ms/step
Epoch 19, Loss: 5.864251358816963e-12, Accuracy: 1.0
1/1 [==============================] - 1s 692ms/step
Epoch 20, Loss: 5.868319285368129e-12, Accuracy: 1.0
1/1 [==============================] - 1s 690ms/step
Epoch 21, Loss: 5.870866293111732e-12, Accuracy: 1.0
1/1 [==============================] - 1s 690

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9696969696969697
f1_score :  0.8767123287671234
accuracy_score :  0.9230769230769231
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [15]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'toothbrush')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

best_params :  {'gamma': 0.001, 'kernel': 'rbf', 'nu': 0.125}
Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_22 (Flatten)        (None, 196608)            0         
                                                                 
 reshape_11 (Reshape)        (None, 16, 16, 768)       0         
                                                                 
 conv2d_transpose_55 (Conv2D  (None, 16, 16, 256)      4915456   
 Transpose)                                                      
                                                                 
 batch_normalization_77 (Bat  (None, 16, 16, 256)      1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_88 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                         

1/1 [==============================] - 0s 469ms/step
Epoch 10, Loss: 1.2903522041553088e-14, Accuracy: 1.0
1/1 [==============================] - 0s 476ms/step
Epoch 11, Loss: 1.2879956737927526e-14, Accuracy: 1.0
1/1 [==============================] - 0s 467ms/step
Epoch 12, Loss: 1.2870360701668082e-14, Accuracy: 1.0
1/1 [==============================] - 0s 487ms/step
Epoch 13, Loss: 1.287455690288878e-14, Accuracy: 1.0
1/1 [==============================] - 0s 486ms/step
Epoch 14, Loss: 1.287883272520652e-14, Accuracy: 1.0
1/1 [==============================] - 0s 468ms/step
Epoch 15, Loss: 1.2885663198893178e-14, Accuracy: 1.0
1/1 [==============================] - 0s 474ms/step
Epoch 16, Loss: 1.289405390726868e-14, Accuracy: 1.0
1/1 [==============================] - 0s 471ms/step
Epoch 17, Loss: 1.2917204164750087e-14, Accuracy: 1.0
1/1 [==============================] - 0s 487ms/step
Epoch 18, Loss: 1.2949004322688855e-14, Accuracy: 1.0
1/1 [==============================] - 0

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.8805555555555555
f1_score :  0.5263157894736842
accuracy_score :  0.7857142857142857
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [16]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'transistor')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 1s 578ms/step
Epoch 8, Loss: 2.6642438255564116e-12, Accuracy: 1.0
1/1 [==============================] - 1s 580ms/step
Epoch 9, Loss: 2.791515282818402e-12, Accuracy: 1.0
1/1 [==============================] - 1s 580ms/step
Epoch 10, Loss: 2.873340888137621e-12, Accuracy: 1.0
1/1 [==============================] - 1s 583ms/step
Epoch 11, Loss: 2.8974958283384256e-12, Accuracy: 1.0
1/1 [==============================] - 1s 591ms/step
Epoch 12, Loss: 2.8805840091711277e-12, Accuracy: 1.0
1/1 [==============================] - 1s 581ms/step
Epoch 13, Loss: 3.072999537126475e-12, Accuracy: 1.0
1/1 [==============================] - 1s 585ms/step
Epoch 14, Loss: 2.959928092557962e-12, Accuracy: 1.0
1/1 [==============================] - 1s 576ms/step
Epoch 15, Loss: 3.1345401534793593e-12, Accuracy: 1.0
1/1 [==============================] - 1s 576ms/step
Epoch 16, Loss: 3.030714134516499e-12, Accuracy: 1.0
1/1 [==============================] - 1s 59

1/1 [==============================] - 1s 596ms/step
Epoch 19, Loss: 3.41017210130079e-12, Accuracy: 1.0
1/1 [==============================] - 1s 581ms/step
Epoch 20, Loss: 3.1459986525594896e-12, Accuracy: 1.0
1/1 [==============================] - 1s 582ms/step
Epoch 21, Loss: 3.4814952570155766e-12, Accuracy: 1.0
1/1 [==============================] - 1s 581ms/step
Epoch 22, Loss: 3.524876137500632e-12, Accuracy: 1.0
1/1 [==============================] - 1s 598ms/step
Epoch 23, Loss: 3.7127380163315404e-12, Accuracy: 1.0
1/1 [==============================] - 1s 580ms/step
Epoch 24, Loss: 3.729132454222128e-12, Accuracy: 1.0
14/14 [==============================] - 25s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.8920833333333333
f1_score :  0.7647058823529412
accuracy_score :  0.76
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [17]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'wood')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 1s 691ms/step
Epoch 3, Loss: 1.0909991883707466e-15, Accuracy: 1.0
1/1 [==============================] - 1s 691ms/step
Epoch 4, Loss: 1.0907608544752129e-15, Accuracy: 1.0
1/1 [==============================] - 1s 692ms/step
Epoch 5, Loss: 1.0932627780431653e-15, Accuracy: 1.0
1/1 [==============================] - 1s 772ms/step
Epoch 6, Loss: 1.0978742371662545e-15, Accuracy: 1.0
1/1 [==============================] - 1s 702ms/step
Epoch 7, Loss: 1.0988792417581717e-15, Accuracy: 1.0
1/1 [==============================] - 1s 686ms/step
Epoch 8, Loss: 1.1023920991486721e-15, Accuracy: 1.0
1/1 [==============================] - 1s 696ms/step
Epoch 9, Loss: 1.1053090688607791e-15, Accuracy: 1.0
1/1 [==============================] - 1s 705ms/step
Epoch 10, Loss: 1.109693734912241e-15, Accuracy: 1.0
1/1 [==============================] - 1s 692ms/step
Epoch 11, Loss: 1.1187565639313884e-15, Accuracy: 1.0
1/1 [==============================] - 1s 689

1/1 [==============================] - 1s 702ms/step
Epoch 15, Loss: 1.145613543469099e-15, Accuracy: 1.0
1/1 [==============================] - 1s 708ms/step
Epoch 16, Loss: 1.1475704013354932e-15, Accuracy: 1.0
1/1 [==============================] - 1s 694ms/step
Epoch 17, Loss: 1.1676027305380574e-15, Accuracy: 1.0
1/1 [==============================] - 1s 702ms/step
Epoch 18, Loss: 1.1777526204658872e-15, Accuracy: 1.0
1/1 [==============================] - 1s 689ms/step
Epoch 19, Loss: 1.1934496232861671e-15, Accuracy: 1.0
1/1 [==============================] - 1s 690ms/step
Epoch 20, Loss: 1.199461333750186e-15, Accuracy: 1.0
1/1 [==============================] - 1s 692ms/step
Epoch 21, Loss: 1.2066280795169047e-15, Accuracy: 1.0
1/1 [==============================] - 1s 685ms/step
Epoch 22, Loss: 1.2121123002130211e-15, Accuracy: 1.0
1/1 [==============================] - 1s 693ms/step
Epoch 23, Loss: 1.2230959881983044e-15, Accuracy: 1.0
1/1 [==============================] - 

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.9956140350877193
f1_score :  0.8823529411764706
accuracy_score :  0.9493670886075949
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [18]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
f_train, f_test = model_predict(X_train, X_test)
clf, ocsvm, sv = OCSVM_SV(f_train, y_tr, X_train)
SV_mask = sv_mask(sv)
fakes, generator = train_gan_data(SV_mask)
save_svmask(fakes, 'zipper')
x_train, y_train = make_dataset(fakes, X_train, y_tr)
f_train, f_test = model_predict(x_train, X_test)
result = OCSVM(f_train, f_test, y_train, y_ts)
res.append(result)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

...

1/1 [==============================] - 0s 483ms/step
Epoch 9, Loss: 1.011407900436577e-18, Accuracy: 1.0
1/1 [==============================] - 0s 471ms/step
Epoch 10, Loss: 1.045127918107301e-18, Accuracy: 1.0
1/1 [==============================] - 0s 485ms/step
Epoch 11, Loss: 1.0423642042831848e-18, Accuracy: 1.0
1/1 [==============================] - 0s 486ms/step
Epoch 12, Loss: 1.038868952604842e-18, Accuracy: 1.0
1/1 [==============================] - 0s 486ms/step
Epoch 13, Loss: 1.0420198903532096e-18, Accuracy: 1.0
1/1 [==============================] - 0s 496ms/step
Epoch 14, Loss: 1.0442636177647597e-18, Accuracy: 1.0
1/1 [==============================] - 0s 487ms/step
Epoch 15, Loss: 1.0431359637946968e-18, Accuracy: 1.0
1/1 [==============================] - 0s 485ms/step
Epoch 16, Loss: 1.0411610700822265e-18, Accuracy: 1.0
1/1 [==============================] - 0s 473ms/step
Epoch 17, Loss: 1.0379644306050153e-18, Accuracy: 1.0
1/1 [==============================] - 0s

1/1 [==============================] - 0s 470ms/step
Epoch 21, Loss: 1.0327083181977002e-18, Accuracy: 1.0
1/1 [==============================] - 0s 469ms/step
Epoch 22, Loss: 1.0344720740588164e-18, Accuracy: 1.0
1/1 [==============================] - 0s 483ms/step
Epoch 23, Loss: 1.0373528339396089e-18, Accuracy: 1.0
1/1 [==============================] - 0s 470ms/step
Epoch 24, Loss: 1.039972825132294e-18, Accuracy: 1.0
13/13 [==============================] - 23s 2s/step


C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.8642331932773109
f1_score :  0.5591397849462365
accuracy_score :  0.7284768211920529
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}


In [19]:
df = pd.DataFrame(res)
df.to_csv('OCSVM_SVGAN_train.csv')

In [23]:
result = OCSVM(f_train, f_test, y_train, y_ts)

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 298, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "C:\Users\Shuta Tsuchio\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'OneClassSVM' object has no attribute 'predict_proba'

  warnings.warn(
C:\Users\Shuta Tsuchio\anaconda3\l

roc_auc :  0.8642331932773109
f1_score :  0.5591397849462365
accuracy_score :  0.7284768211920529
best_params :  {'gamma': 0.0001, 'kernel': 'rbf', 'nu': 0.125}
